## Langchain + OpenAI LLM tracing using Maxim + Langchain

In [ ]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

In [2]:
import os

LOG_REPOSITORY_ID = os.getenv("MAXIM_LOG_REPO_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from maxim import Maxim

logger = Maxim().logger()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
from maxim.logger.langchain import MaximLangchainTracer

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(
    model=MODEL_NAME, api_key=OPENAI_API_KEY
)  # Changed streaming to False

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("human", "{input}")]
)

# Create a chain using pipe
chain = prompt | llm | JsonOutputParser()

# Run the chain with callbacks for tracing
user_input = (
    "give me a basic json response. dont add anything else, just the json. not even ```"
)
response = ""
for chunk in chain.stream(
    {"input": user_input}, config={"callbacks": [langchain_tracer]}
):
    pass